# Download unified dataset

In [12]:
%load_ext watermark
%watermark -v -m  -u -n -p pandas,numpy,scanpy,requests,json -a Filippo_Valle -g -r -b -w

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Filippo_Valle 
last updated: Thu Jun 18 2020 

CPython 3.7.6
IPython 7.15.0

pandas 1.0.4
numpy 1.18.5
scanpy 1.5.1
requests 2.23.0
json 2.0.9

compiler   : GCC 7.5.0
system     : Linux
release    : 4.19.76-linuxkit
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit
Git hash   : 3c8ae8f7082ca41e4f98ab084dc707834cbe547b
Git repo   : git@github.com:fvalle1/phd.git
Git branch : master
watermark 2.0.2


In [ ]:
# import libraries
import pandas as pd
import numpy as np
import scanpy as sc
import requests
import json
import os, sys

## Download data
Download data from [https://doi.org/10.6084/m9.figshare.5330593](https://doi.org/10.6084/m9.figshare.5330593) and [https://figshare.com/articles/Data_record_2/5330575](https://figshare.com/articles/Data_record_2/5330575) and set working_dir appropriately

In [ ]:
#path/to/files/downloaded/
working_dir = "/home/jovyan/work/phd/datasets/merged/"
os.chdir(working_dir)

In [ ]:
files = filter(lambda f: "fpkm" in f, os.listdir("data"))

Download GTEx metadata from[http://gtexportal.org](http://gtexportal.org)

In [ ]:
df_gtex=pd.read_csv("https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt", sep='\t', index_col=0)

Download TCGA metadata using gdc API

In [ ]:
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "cases.project.program.name",
            "value": ["TCGA"]
            }
        }
        
    ]
}
params = {
    "filters": json.dumps(filters),
    "fields": "primary_site,disease_type,submitter_id,project.project_id",
    "format": "TSV",
    "size": "10000000"
    }
response = requests.get("https://api.gdc.cancer.gov/cases", headers = {"Content-Type": "application/json"}, params = params)
with open("files.txt","w") as file:
    file.write(response.content.decode("utf-8"))
df_tcga = pd.read_csv("files.txt", sep='\t').set_index("submitter_id")

Create an empty DataFrame to fulfill with unified dataset

In [ ]:
df=pd.DataFrame()

In [ ]:
for file in files:
    df = df.append(pd.read_csv("data/%s"%file, sep='\t', index_col=0).drop('Entrez_Gene_Id',1).transpose(), sort=True)
df = df.transpose()
df = df.dropna(how='any', axis=0) # drop genes not always determined

Prepare a metadata file

In [ ]:
df_files = pd.DataFrame(index=df.columns)

We define useful function to discriminate samples coming from GTEx and TCGA

In [ ]:
def get_site(file):
    '''
    Returns GTEX or TCGA primary site
    '''
    if ("df_gtex" not in globals().keys()) or ("df_tcga" not in globals().keys()):
        raise NotImplementedError("Please define datasets with gtex and tcga metadata")
    if 'GTEX' in file:
        return df_gtex.at[file, 'SMTS']
    if 'TCGA' in file:
        return df_tcga.at[file[:12],'primary_site']

def get_source(file):
    if 'GTEX' in file:
        return 'gtex'
    if 'TCGA' in file:
        return 'tcga'

#https://www.researchgate.net/post/How_can_I_get_the_normal_sample_of_TCGA_data
def get_status(file):
    if 'GTEX' in file:
        return 'healthy'
    if 'TCGA' in file:
        if (file[13:15]=="11") or (file[13:15]=="10"):
            return 'healthy'
        else:
            return "tumor"

In [ ]:
df_files.insert(0, 'primary_site', [get_site(file) for file in df.columns])
df_files.insert(1, 'dataset', [get_source(file) for file in df.columns])
df_files.insert(1, 'status', [get_status(file) for file in df.columns])
df_files.groupby(["dataset","status"]).count()

Save metadata file

In [ ]:
df_files.to_csv("files.dat", index=True, header=True)

## Split / shuffle and select

In [ ]:
df_files = pd.read_csv("files.dat", index_col=0)
df.head(2)

Unify names in different datasets

In [ ]:
df_files.replace('Uterus, NOS', 'Uterus', inplace=True)
df_files.replace('Bronchus and lung', 'Lung', inplace=True)
df_files.replace('Liver and intrahepatic bile ducts', 'Liver', inplace=True)
df_files.replace('Prostate gland', 'Prostate', inplace=True)
df_files.replace('Thyroid gland', 'Thyroid', inplace=True)
df_files.replace('Base of Tongue', 'Salivary Gland', inplace=True)
df_files.replace('Bones, joints and articular cartilage of other and unspecified sites', 'Salivary Gland', inplace=True)
df_files.replace('Floor of mouth', 'Salivary Gland', inplace=True)
df_files.replace('Gum', 'Salivary Gland', inplace=True)
df_files.replace('Hypopharynx', 'Salivary Gland', inplace=True)
df_files.replace('Larynx', 'Salivary Gland', inplace=True)
df_files.replace('Lip', 'Salivary Gland', inplace=True)
df_files.replace('Oropharynx', 'Salivary Gland', inplace=True)
df_files.replace('Other and ill-defined sites in lip, oral cavity and pharynx', 'Salivary Gland', inplace=True)
df_files.replace('Other and unspecified parts of mouth', 'Salivary Gland', inplace=True)
df_files["tissue_hd"]=df_files["primary_site"]+"_"+df_files["dataset"]
df_files['primary_site'].unique()

In [ ]:
samples = df_files[df_files["primary_site"]=="Lung"]

In [ ]:
df[df.columns[df.columns.isin(samples.index)]].to_csv("mainTable_merged.csv", index=True, header=True)

In [ ]:
df_files.to_csv("files.dat", index=True, header=True)

# Use scanpy to filter HVG

In [ ]:
adata = sc.AnnData(df[samples.index].transpose(), obs=samples)
adata_log = sc.pp.log1p(adata, copy=True)

In [ ]:
sc.pp.highly_variable_genes(adata_log, n_top_genes=3000, n_bins=50)

In [ ]:
sc.pl.highly_variable_genes(adata_log, log=True, save='hvg.pdf')

In [ ]:
hvg = adata_log.var[adata_log.var['highly_variable']==True].index
samples = adata_log.obs.index

# Run hierarchical Stochastic Block Model

In [ ]:
from sbmtm import sbmtm

In [ ]:
model = sbmtm()

In [ ]:
model.make_graph_from_BoW_df(df.loc[hvg, samples])

In [ ]:
model.save_graph("graph.xml.gz")